### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [1]:
import json
import nltk

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [5]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 500,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30218
Seconds required: 2.824

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 295773.733269
Feature norm: 1.000000
Error norm: 81338.520474
Active features: 30059
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.101

***** Iteration #2 *****
Loss: 293961.287058
Feature norm: 1.004048
Error norm: 78508.957996
Active features: 28477
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.551

***** Iteration #3 *****
Loss: 262530.250355
Feature norm: 1.249336
Error norm: 112881.356688
Active features: 20932
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #44 *****
Loss: 42337.834142
Feature norm: 38.664916
Error norm: 10076.319691
Active features: 25405
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #45 *****
Loss: 41877.452258
Feature norm: 39.718802
Error norm: 7920.058188
Active features: 25374
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.577

***** Iteration #46 *****
Loss: 41158.149954
Feature norm: 42.117048
Error norm: 15437.131370
Active features: 25209
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #47 *****
Loss: 40651.086292
Feature norm: 44.004180
Error norm: 4745.305950
Active features: 24877
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #48 *****
Loss: 40203.240422
Feature norm: 45.793397
Error norm: 3359.640481
Active features: 24618
Line search trials: 1
Line search step: 1.0000

***** Iteration #86 *****
Loss: 33896.477123
Feature norm: 94.781236
Error norm: 4143.768261
Active features: 22124
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #87 *****
Loss: 33863.463724
Feature norm: 95.098983
Error norm: 3739.602864
Active features: 22146
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.582

***** Iteration #88 *****
Loss: 33831.098907
Feature norm: 95.582780
Error norm: 6427.508112
Active features: 22078
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.580

***** Iteration #89 *****
Loss: 33803.303186
Feature norm: 96.055066
Error norm: 6845.138048
Active features: 22027
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.550

***** Iteration #90 *****
Loss: 33773.941436
Feature norm: 96.683014
Error norm: 4812.431004
Active features: 22014
Line search trials: 1
Line search step: 1.000000

***** Iteration #131 *****
Loss: 33226.017936
Feature norm: 110.618474
Error norm: 3112.265806
Active features: 21444
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.587

***** Iteration #132 *****
Loss: 33220.959186
Feature norm: 110.928192
Error norm: 5181.473985
Active features: 21401
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.569

***** Iteration #133 *****
Loss: 33211.329210
Feature norm: 111.124606
Error norm: 2884.181422
Active features: 21407
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.609

***** Iteration #134 *****
Loss: 33205.884082
Feature norm: 111.401127
Error norm: 4915.024824
Active features: 21391
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.594

***** Iteration #135 *****
Loss: 33197.189767
Feature norm: 111.571433
Error norm: 3243.210315
Active features: 21392
Line search trials: 1
Line search step

***** Iteration #173 *****
Loss: 32979.063920
Feature norm: 118.377931
Error norm: 3605.891637
Active features: 21152
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.573

***** Iteration #174 *****
Loss: 32972.991609
Feature norm: 118.594537
Error norm: 3830.271427
Active features: 21141
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #175 *****
Loss: 32966.630439
Feature norm: 118.717074
Error norm: 2859.691392
Active features: 21158
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.580

***** Iteration #176 *****
Loss: 32960.458596
Feature norm: 118.882790
Error norm: 2984.133718
Active features: 21171
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.591

***** Iteration #177 *****
Loss: 32954.272015
Feature norm: 119.005138
Error norm: 2751.751738
Active features: 21177
Line search trials: 1
Line search step

...

***** Iteration #296 *****
Loss: 32601.343147
Feature norm: 126.740298
Error norm: 1963.154537
Active features: 20286
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.550

***** Iteration #297 *****
Loss: 32599.354354
Feature norm: 126.749795
Error norm: 1373.065842
Active features: 20293
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.545

***** Iteration #298 *****
Loss: 32597.721413
Feature norm: 126.781513
Error norm: 1967.593813
Active features: 20284
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.543

***** Iteration #299 *****
Loss: 32595.848563
Feature norm: 126.797831
Error norm: 1701.452787
Active features: 20281
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #300 *****
Loss: 32594.488003
Feature norm: 126.842771
Error norm: 2233.406375
Active features: 20287
Line search trials: 1
Line search step

***** Iteration #341 *****
Loss: 32538.989273
Feature norm: 127.918764
Error norm: 971.963016
Active features: 20076
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.597

***** Iteration #342 *****
Loss: 32538.725378
Feature norm: 127.949820
Error norm: 2047.186284
Active features: 20082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.578

***** Iteration #343 *****
Loss: 32537.025445
Feature norm: 127.964159
Error norm: 859.666216
Active features: 20084
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.603

***** Iteration #344 *****
Loss: 32536.412830
Feature norm: 127.992761
Error norm: 1847.633920
Active features: 20082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.576

***** Iteration #345 *****
Loss: 32535.084359
Feature norm: 128.007415
Error norm: 1048.655025
Active features: 20078
Line search trials: 1
Line search step: 

***** Iteration #386 *****
Loss: 32496.985366
Feature norm: 128.685816
Error norm: 2063.842244
Active features: 19812
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.576

***** Iteration #387 *****
Loss: 32495.466156
Feature norm: 128.688562
Error norm: 681.936062
Active features: 19802
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #388 *****
Loss: 32494.809197
Feature norm: 128.706750
Error norm: 1678.681387
Active features: 19800
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.579

***** Iteration #389 *****
Loss: 32493.255833
Feature norm: 128.712132
Error norm: 580.791894
Active features: 19785
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.567

***** Iteration #390 *****
Loss: 32492.755250
Feature norm: 128.733128
Error norm: 2129.328662
Active features: 19779
Line search trials: 1
Line search step: 

***** Iteration #425 *****
Loss: 32459.250852
Feature norm: 129.251591
Error norm: 1082.284004
Active features: 19658
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #426 *****
Loss: 32458.772129
Feature norm: 129.271514
Error norm: 2022.000962
Active features: 19639
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #427 *****
Loss: 32457.402026
Feature norm: 129.282147
Error norm: 873.182930
Active features: 19626
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.576

***** Iteration #428 *****
Loss: 32456.776042
Feature norm: 129.297938
Error norm: 1740.097663
Active features: 19624
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #429 *****
Loss: 32455.653972
Feature norm: 129.304110
Error norm: 825.261702
Active features: 19614
Line search trials: 1
Line search step: 

***** Iteration #466 *****
Loss: 32426.935421
Feature norm: 129.636999
Error norm: 2103.956992
Active features: 19530
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.565

***** Iteration #467 *****
Loss: 32425.219221
Feature norm: 129.634520
Error norm: 572.989481
Active features: 19534
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.556

***** Iteration #468 *****
Loss: 32424.720677
Feature norm: 129.645361
Error norm: 1398.221413
Active features: 19523
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.572

***** Iteration #469 *****
Loss: 32423.871116
Feature norm: 129.648798
Error norm: 969.391653
Active features: 19512
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.610

***** Iteration #470 *****
Loss: 32423.645351
Feature norm: 129.666782
Error norm: 1823.288420
Active features: 19503
Line search trials: 1
Line search step: 

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

治 (O)
以 (O)
疏 (O)
肝 (O)
泄 (O)
热 (O)
、 (O)
和 (O)
胃 (O)
止 (O)
痛 (O)


In [12]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    304100
        B-S       0.61      0.45      0.52      4540
        I-S       0.50      0.32      0.39      5135
        E-S       0.62      0.46      0.53      4528

avg / total       0.96      0.96      0.96    318303

